In [ ]:
import requests
from requests import Response
from io import StringIO
from csv import DictReader
def get_aqi_data():
    try:
        aqi_url='https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=CSV'
        response:Response=requests.get(aqi_url)
        response.raise_for_status()
    except Exception as e:
        print(f'HTTP error occurred:{e}')
        return None
    else:
        if response.status_code != 200:
            print('Status code error.')
            return None
    if response:
        rawString=response.text
    file:StringIO=StringIO(rawString)
    read:DictReader=DictReader(file)
    data:list[dict]=list(read)
    return data
aqi_data=get_aqi_data()

In [ ]:
from pydantic import RootModel,BaseModel,Field,field_validator
class Site(BaseModel):
    county:str
    sitename:str
    aqi:float
    creat_time:str=Field(alias='datacreationdate')
    particulate_matter:float=Field(alias='pm2.5')
    @field_validator('particulate_matter',mode='before')
    @classmethod
    def empty_string_pm25(cls,value):
        if value=='':
            return 0.0
        else:
            return value
class Sites(RootModel):
    root:list[Site]

    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]
    
sites:Sites=Sites.model_validate(aqi_data)
download:list[dict]=sites.model_dump()#RootModel to list

In [ ]:
import pandas as pd
pd.DataFrame(download)